## generate narrative from resource

### import modules

In [1]:
import os, sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
# This allows you to import the desired function from the module hierarchy:
from fhirclient.r4models.fhirabstractbase import FHIRValidationError
from fhirclient.r4models import capabilitystatement as CS
from fhirclient.r4models import narrative as N
from json import dumps, loads, load, JSONDecodeError
from requests import get, post, put
from pathlib import Path
from IPython.display import display as Display, HTML, Markdown
from pprint import pprint
from collections import namedtuple
from jinja2 import Environment, FileSystemLoader, select_autoescape
from stringcase import snakecase, titlecase
from itertools import zip_longest
from commonmark import commonmark
from lxml import etree

### Define variables

#### get CapStatement as json

(todo add a graphic to show where to upload the file)

In [2]:
fname = input('enter the CapStatment file name for the file in the "cs_source_file" directory:')

enter the CapStatment file name for the file in the "cs_source_file" directory: input.json


In [3]:
path = Path.cwd() / 'cs_source_file'/ fname
try: cs_json = path.read_text()
except FileNotFoundError:
       path = Path.cwd() / 'cs_source_file'/ f'{fname}.json'
       cs_json = path.read_text() 
cs_py = CS.CapabilityStatement(loads(cs_json))
print(cs_py.id)
# sp_map = {sp.code:sp.type for sp in df_sp.itertuples(index=True)}
# pname_map = {p.Profile:p.Name for p in df_profiles.itertuples(index=True)}

# this is wrong if for most IGs should be supportedProfiles not Profiles
pname_map = {r.profile:titlecase(r.profile.split('/')[-1]) for r in cs_py.rest[0].resource if r.profile}
#add in supported Profiles too
spname_map = {sp:titlecase(sp.split('/')[-1]) for \
              r in cs_py.rest[0].resource if r.supportedProfile for sp in r.supportedProfile}
#flatten_matrix = [val for sublist in matrix for val in sublist] 

# Adding elements from dict2 to dict1
pname_map.update(spname_map)

sp_map = None

print(pname_map)

carin-bb-capabilitystatement-healthplan-api
{'http://hl7.org/fhir/us/carin/StructureDefinition/carin-bb-patient': 'Carin Bb Patient', 'http://hl7.org/fhir/us/carin/StructureDefinition/carin-bb-coverage': 'Carin Bb Coverage', 'http://hl7.org/fhir/us/carin/StructureDefinition/carin-bb-explanationofbenefit': 'Carin Bb Explanationofbenefit', 'http://hl7.org/fhir/us/carin/StructureDefinition/carin-bb-relatedperson': 'Carin Bb Relatedperson', 'http://hl7.org/fhir/us/carin/StructureDefinition/carin-bb-practitionerrole': 'Carin Bb Practitionerrole', 'http://hl7.org/fhir/us/carin/StructureDefinition/carin-bb-organization': 'Carin Bb Organization', 'http://hl7.org/fhir/us/carin/StructureDefinition/carin-bb-explanationofbenefit-inpatient-facility': 'Carin Bb Explanationofbenefit Inpatient Facility', 'http://hl7.org/fhir/us/carin/StructureDefinition/carin-bb-explanationofbenefit-outpatient-facility': 'Carin Bb Explanationofbenefit Outpatient Facility', 'http://hl7.org/fhir/us/carin/StructureDefini

### get pathmap from package file
- Get spec_internal from package a json file which includes canonical to local relative page links

(todo add a graphic to show where to find this file - could eliminate above step too...)

In [4]:
ig_package = input("Get NPM package url from the IG Publication (Version) History page:")

Get NPM package url from the IG Publication (Version) History page: http://build.fhir.org/ig/HL7/carin-bb/package.tgz


In [5]:
r = get(ig_package)
r.status_code
#write to file
path = Path.cwd() /  'cs_source_file' / ig_package.split('/')[-1]
path.write_bytes(r.content)

258951

In [6]:
import tarfile

def get_si(tar_file):
    print(path)
    with tarfile.open(tar_file, mode='r') as tf:
        #pprint(tf.getnames())
        f = tf.extractfile('package/other/spec.internals')
        r = f.read()
        return(loads(r))


si = get_si(path)
       
path_map = si['paths']
path_map

/Users/ehaas/Documents/Python/MyBinder/CapStatement/cs_source_file/package.tgz


{'http://hl7.org/fhir/us/carin-bb/ImplementationGuide/hl7.fhir.us.carin-bb|0.1.0': '0.1.0/ImplementationGuide-hl7.fhir.us.carin-bb.html',
 'http://hl7.org/fhir/us/carin-bb/ImplementationGuide/hl7.fhir.us.carin-bb': 'ImplementationGuide-hl7.fhir.us.carin-bb.html',
 'http://hl7.org/fhir/us/carin/StructureDefinition/carin-bb-explanationofbenefit': 'StructureDefinition-CARIN-BB-ExplanationOfBenefit.html',
 'http://hl7.org/fhir/us/carin/StructureDefinition/carin-bb-explanationofbenefit|0.1.0': '0.1.0/StructureDefinition-CARIN-BB-ExplanationOfBenefit.html',
 'http://hl7.org/fhir/us/carin-bb/StructureDefinition/CARIN-BB-ExplanationOfBenefit': 'StructureDefinition-CARIN-BB-ExplanationOfBenefit.html',
 'http://hl7.org/fhir/us/carin/StructureDefinition/carin-bb-explanationofbenefit-outpatient-facility': 'StructureDefinition-CARIN-BB-ExplanationOfBenefit-Outpatient-Facility.html',
 'http://hl7.org/fhir/us/carin/StructureDefinition/carin-bb-explanationofbenefit-outpatient-facility|0.1.0': '0.1.0/S

### Get URL:Name/Title mapping from package file

In [7]:
from json import JSONDecodeError
def get_name_map(tar_file):
    print(path)
    name_map = {}
    with tarfile.open(tar_file, mode='r') as tf:
        for p_file in tf.getnames():
            #print(p_file)
            f = tf.extractfile(p_file)
            r = f.read()
            #print(type(r))
            try:
                r_dict = loads(r)
            except JSONDecodeError: # not a json file
                continue
            try:
                name_map[r_dict['url']] = r_dict['title']
                print(r_dict['title'])
            except KeyError:
                try:
                    name_map[r_dict['url']] = r_dict['name']
                    print(r_dict['name'])
                except KeyError: # not a conf resource
                    print(f'****{p_file}is not a conf resource***')
        return(name_map)


name_map = get_name_map(path)
name_map       

/Users/ehaas/Documents/Python/MyBinder/CapStatement/cs_source_file/package.tgz
Consumer-Directed Payer Data Exchange
Consumer-Directed Payer Data Exchange
Consumer-Directed Payer Data Exchange
CARIN Blue Button Explanation of Benefit Profile
CARIN Blue Button Explanation of Benefit Outpatient Facility Profile
CARIN Blue Button Claim Information Category Code System
CARIN Blue Button Explanation of Benefit Pharmacy Profile
CARIN Blue Button Related Claim Relationship Value Set
CARIN Blue Button Explanation of Benefit Inpatient Facility Profile
CARIN Blue Button Claim Type Value Set
CARIN Blue Button Claim Type Code System
CARIN Blue Button Related Claim Relationship Code System
CARIN Blue Button Claim Care Team Role Value Set
CARIN Blue Button Claim Care Team Role Code System
CARIN Blue Button Diagnosis Type Value Set
CARIN Blue Button Revenue Center Value Set
CARIN Blue Button Adjudication Amount Category Value Set
CARIN Blue Button Claim Adjudication Category Code System
CARIN Blue Bu

{'https://build.fhir.org/ig/HL7/carin-bb/': 'Consumer-Directed Payer Data Exchange',
 'http://hl7.org/fhir/us/carin-bb/ImplementationGuide/hl7.fhir.us.carin-bb': 'Consumer-Directed Payer Data Exchange',
 'http://hl7.org/fhir/us/carin/StructureDefinition/carin-bb-explanationofbenefit': 'CARIN Blue Button Explanation of Benefit Profile',
 'http://hl7.org/fhir/us/carin/StructureDefinition/carin-bb-explanationofbenefit-outpatient-facility': 'CARIN Blue Button Explanation of Benefit Outpatient Facility Profile',
 'http://hl7.org/fhir/us/carin/CodeSystem/carin-bb-claiminformationcategory': 'CARIN Blue Button Claim Information Category Code System',
 'http://hl7.org/fhir/us/carin/StructureDefinition/carin-bb-explanationofbenefit-pharmacy': 'CARIN Blue Button Explanation of Benefit Pharmacy Profile',
 'http://hl7.org/fhir/us/carin/ValueSet/carin-bb-relatedclaimrelationship': 'CARIN Blue Button Related Claim Relationship Value Set',
 'http://hl7.org/fhir/us/carin/StructureDefinition/carin-bb-ex

### Get SearchParameter parameter name: type mapping from package file

In [8]:
def get_sp_map(tar_file):
    print(path)
    sp_map = {}
    with tarfile.open(tar_file, mode='r') as tf:
        for p_file in tf.getnames():
            #print(p_file)
            if 'searchparameter' in p_file.lower():
                print(f'**********{p_file}')
                f = tf.extractfile(p_file)
                r = f.read()
                #print(type(r))
                try:
                    r_dict = loads(r)
                except JSONDecodeError: # not a json file
                    continue
                try:
                    sp_map[r_dict['code']] = r_dict['type']
                    print(r_dict['type'])
                except KeyError:
                    print(f'{p_file} has no type')
        return(sp_map)


sp_map = get_sp_map(path)
sp_map       

/Users/ehaas/Documents/Python/MyBinder/CapStatement/cs_source_file/package.tgz
**********package/SearchParameter-carin-bb-searchparameter-explanationofbenefit-created.json
date
**********package/SearchParameter-carin-bb-searchparameter-coverage-patient.json
reference
**********package/SearchParameter-carin-bb-searchparameter-explanationofbenefit-patient.json
reference


{'status': 'date', 'patient': 'reference'}

### render using Jinja2 templates

In [9]:
in_path = ''
in_file = 'R4capabilitystatement-server.j2'

def markdown(text, *args, **kwargs):
    return commonmark(text, *args, **kwargs)

env = Environment(
    loader=FileSystemLoader(searchpath = in_path),
    autoescape=select_autoescape(['html','xml','xhtml','j2','md'])
    )

env.filters['markdown'] = markdown

template = env.get_template(in_file)

rendered = template.render(cs=cs_py, path_map=path_map, pname_map=name_map, sp_map=sp_map )

display(HTML(rendered))


parser = etree.XMLParser(remove_blank_text=True)
root = etree.fromstring(rendered, parser=parser)

div = (etree.tostring(root[1][0], encoding='unicode', method='html'))
narr = N.Narrative()
narr.status = 'generated'
narr.div = div
cs_py.text = narr


#print(dumps(cs.as_json(),indent=3))

### Save to local dir

In [10]:
# save to file
ig_source_path = ''
print('...........saving to file............')
#save in ig_source folder
path = Path.cwd() / ig_source_path / 'resources' / f'capabilitystatement-{cs_py.id.lower()}.json'
path.write_text(dumps(cs_py.as_json(), indent=4))

...........saving to file............


23471